## GPU Programming: Perform vector and matrix operations in CUDA

### Section1: Compile and Run CUDA Code

We will implment a program to add two vectors in cuda. 
The following code snippt is from `example_vector_add.cu`. 

```cpp

__global__ void VecAddKernel(int* A, int* B, int* C, int n) {
  // blockDim is size of block along x-axis
  // blockIdx is the index of the current thread's block
  // threadIdx is the index of the current thread within the block
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  if (i < n) {
    C[i] = A[i] + B[i];
  }
}

```

Please check the full code in `example_vector_add.cu`. 

Run the following command: 

In [3]:
# Remove ! if running on the terminal of your vm
# Compile the codes for matrix addition
!nvcc -o vecadd example_vector_add.cu

Run the following command to check the result: 

In [4]:
!./vecadd

Vector add verified! COMPLETED SUCCESSFULLY


In `example_matadd.cu` and `example_matmul2.cu`, there are example codes for matrix addition and matrix multiplication. 

The following code snippets are from `example_matadd.cu` to perform matrix multiplication:

```cpp

__global__ void matrixAdd(const int * a, const int * b,
                          int * c, int N) {
  // Compute each thread's global row and column index
  int row = blockIdx.x * blockDim.x + threadIdx.x;
  int col = blockIdx.y * blockDim.y + threadIdx.y;
  
  // Iterate over row, and down column
  if (row < N && col < N) {
    c[row * N + col] = a[row * N + col] + b[row * N + col];
  }
}

```

Run the following command to check the results.

In [5]:
# Remove ! if running on the terminal of your vm
# Compile the codes for matrix addition
!nvcc -o matadd example_matadd.cu

In [6]:
# Run the codes
!./matadd

COMPLETED SUCCESSFULLY


The following code snippets illustrate the multiplication of two matrices. 

```c++

__global__ void MatmulKernel(const float* a, const float* b, float* out, 
                             int M, int N, int P) {
  // Compute each thread's global row and column index
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
  if (idx >= M * P) return;
  int row = idx / P;
  int col = idx % P;
  if (row < M && col < P) {
    // Calculate the matrix multiplication for row in matrix a and col in matrix b
    float sum = 0.0;
    for (int i = 0; i < N; i++) {
      sum += a[row * N + i] * b[i * P + col];
    }
    out[row * P + col] = sum;
  }
}

```

```c++
__global__ void MatmulKernel(const int *a, const int *b, int *c, int M, int N, int P) {
  // Compute each thread's global row and column index
  int row = blockIdx.x * blockDim.x + threadIdx.x;
  int col = blockIdx.y * blockDim.y + threadIdx.y;
  if (row >= M || col >= P) return;
  // Iterate over row, and down column
  c[row * P + col] = 0;
  for (int k = 0; k < N; k++) {
    // Accumulate results for a single element
    c[row * P + col] += a[row * N + k] * b[k * P + col];
  }
}
```

Run the following command. 

In [7]:
# Compile the codes for matrix addition
!nvcc -o matmul example_matmul2.cu

In [8]:
# Run the codes
!./matmul

COMPLETED SUCCESSFULLY


### Section2: Run CUDA Codes with Python Calls

This section shows three demos, including the vector addition, window summation and matrix multiplication implemented in CUDA. We call these CUDA functions within the python codes, which follows the same recipe in our assignment1.

## CUDA Example for Vector Add

We demonstrate how we can call C function in python file with ctypes with `VecAddCPU` function.

We also demonstrate two ways to write CUDA codes which can be called in python functions. The difference between them is that we create CUDA memory and copy the data to CUDA device by `pycuda` package in the `VecAddCUDA` function and `cudaMemcpy` in cpp codes in `VecAddCUDA2` function.

We use `pycuda` in assignment1 to call CUDA kernel functions.

In [9]:
!nvcc -o vector_add.so --shared example_vector_add.cu -Xcompiler -fPIC

```python
# Define the argument types and return types
lib.VecAddCUDA.argtypes = [
    ctypes.POINTER(ctypes.c_int),
    ctypes.POINTER(ctypes.c_int),
    ctypes.POINTER(ctypes.c_int),
    ctypes.c_int,
]

lib.VecAddCUDA.restype = None

# Load the arrays to CUDA device
a_gpu = gpuarray.to_gpu(a)
b_gpu = gpuarray.to_gpu(b)
c_gpu = gpuarray.to_gpu(cgpu)

# Call the C wrapper function with CUDA kernel
lib.VecAddCUDA(
    ctypes.cast(a_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
    ctypes.cast(b_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
    ctypes.cast(c_gpu.ptr, ctypes.POINTER(ctypes.c_int)),
    ctypes.c_int(size)
)

# Load the gpuarray back to array in the host device
cgpu = c_gpu.get()
print(f"After offload: {cgpu}, {type(cgpu)}")
```

In [10]:
!python test_vector_add.py

Input a: [9 7 1 4 1 4 2 1 6 1]
Input b: [8 7 6 8 8 7 1 2 9 2]
Numpy add: [17 14  7 12  9 11  3  3 15  3], <class 'numpy.ndarray'>
CPU add: [17 14  7 12  9 11  3  3 15  3], <class 'numpy.ndarray'>
GPU add: [17 14  7 12  9 11  3  3 15  3], <class 'pycuda.gpuarray.GPUArray'>
After offload: [17 14  7 12  9 11  3  3 15  3], <class 'numpy.ndarray'>
GPU add2: [17 14  7 12  9 11  3  3 15  3], <class 'numpy.ndarray'>


## CUDA Example for Window Sum

Demo of Window Sum to get to know about synchronization in CUDA.

In [11]:
!nvcc -o window_sum.so --shared example_window_sum.cu -Xcompiler -fPIC

In [12]:
!python test_window_sum.py

Input: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
Numpy window sum: [15. 20. 25. 30. 35. 40. 45. 50.]
GPU simple window sum: [15. 20. 25. 30. 35. 40. 45. 50.]
GPU shared window sum: [15. 20. 25. 30. 35. 40. 45. 50.]


## CUDA Example for Matrix Multiplication

Demo of matrix multiplication.

In [13]:
!nvcc -o matmul.so --shared example_matmul.cu -Xcompiler -fPIC

In [14]:
!python test_matmul.py

Input a: [[1. 2. 2. 1.]
 [2. 2. 2. 2.]
 [2. 2. 2. 1.]
 [1. 2. 1. 1.]]
Input b: [[1. 1.]
 [1. 2.]
 [1. 2.]
 [2. 1.]]
Numpy matmul: [[ 7. 10.]
 [10. 12.]
 [ 8. 11.]
 [ 6.  8.]], <class 'numpy.ndarray'>
GPU matmul: [[ 7. 10.]
 [10. 12.]
 [ 8. 11.]
 [ 6.  8.]], <class 'pycuda.gpuarray.GPUArray'>
After offload: [[ 7. 10.]
 [10. 12.]
 [ 8. 11.]
 [ 6.  8.]], <class 'numpy.ndarray'>
Compare result: 0.0
